In [16]:
# Importation des bibliothèques nécessaires pour le projet
import tensorflow as tf       # Framework de deep learning
import matplotlib.pyplot as plt  # Visualisation de graphiques
import numpy as np            # Calculs numériques et manipulation de tableaux
import os                     # Gestion des fichiers et répertoires
import seaborn as sns         # Visualisation statistique avancée
import pandas as pd           # Manipulation de données tabulaires

In [17]:
# Définition des chemins vers les répertoires d'entraînement et de test
train_dir = "Data/Training"
test_dir = "Data/Testing"

# Affichage du nombre d'images par classe dans le dataset d'entraînement
for classe in os.listdir(train_dir):
    print(classe, len(os.listdir(os.path.join(train_dir, classe))))

glioma 1321
meningioma 1339
notumor 1595
pituitary 1457


In [18]:
# Définition des hyperparamètres pour le chargement des données
IMG_SIZE = (224, 224)  # Taille des images (format attendu par ResNet50)
BATCH_SIZE = 32        # Nombre d'images à traiter simultanément
SEED = 42              # Graine aléatoire pour la reproductibilité des résultats

In [19]:
# Création du dataset d'entraînement (80% des données du répertoire Training)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # 20% des données pour la validation
    subset="training",      # On récupère la partie "entraînement"
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Création du dataset de validation (20% des données du répertoire Training)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # 20% des données pour la validation
    subset="validation",    # On récupère la partie "validation"
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Found 5712 files belonging to 4 classes.
Using 1142 files for validation.


In [20]:
# Création du dataset de test à partir du répertoire Testing
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False  # Pas de mélange pour maintenir l'ordre lors de l'évaluation
)

Found 1311 files belonging to 4 classes.


In [21]:
# Affichage des noms des classes détectées dans le dataset
print(train_ds.class_names)

['glioma', 'meningioma', 'notumor', 'pituitary']


In [22]:
# Normalisation des pixels des images de [0, 255] à [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Application de la normalisation aux trois datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [23]:
# Création d'un pipeline d'augmentation de données pour enrichir le dataset d'entraînement
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),  # Retournement horizontal aléatoire
    tf.keras.layers.RandomRotation(0.1),       # Rotation aléatoire jusqu'à 10% (36°)
    tf.keras.layers.RandomZoom(0.1),           # Zoom aléatoire jusqu'à 10%
])

In [24]:
# Application de l'augmentation de données uniquement au dataset d'entraînement
# (pas aux datasets de validation et test pour évaluer les performances réelles)
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)

In [25]:
# Optimisation du chargement des données avec prefetch
# AUTOTUNE permet à TensorFlow de déterminer automatiquement le nombre optimal d'éléments à précharger
AUTOTUNE = tf.data.AUTOTUNE

# Application du prefetch pour charger les données en avance pendant l'entraînement
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

## Tache 2

In [26]:
# Chargement du modèle ResNet50 pré-entraîné sur ImageNet
# Il sera utilisé comme extracteur de caractéristiques (feature extractor)
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",      # Poids pré-entraînés sur ImageNet
    include_top=False,       # Exclusion de la couche de classification finale
    pooling="avg",           # Pooling moyen global pour obtenir un vecteur de features
    input_shape=(224,224,3)  # Taille d'entrée standard pour ResNet50
)

In [27]:
def extract_features(dataset):
    """
    Extrait les caractéristiques des images en utilisant le modèle de base pré-entraîné.
    
    Cette fonction parcourt toutes les images du dataset et utilise le modèle base_model
    pour extraire les features (caractéristiques) de chaque image. Les features sont
    des représentations vectorielles des images obtenues par le réseau convolutionnel.
    
    Args:
        dataset: Un tf.data.Dataset contenant des tuples (images, labels) où
                 - images: batch d'images à traiter (tenseur TensorFlow)
                 - labels: batch d'étiquettes correspondantes (tenseur TensorFlow)
    
    Returns:
        tuple: Un tuple (X, y) où
            - X: array numpy de shape (n_samples, n_features) contenant les features extraites
            - y: array numpy de shape (n_samples,) contenant les labels correspondants
    """
    X = []
    y = []
    
    for images, labels in dataset:
        features = base_model.predict(images, verbose=0)
        X.append(features)
        y.append(labels.numpy())
        
    return np.vstack(X), np.concatenate(y)

In [28]:
# Extraction des caractéristiques (features) pour les trois datasets
# Ces features seront utilisées pour entraîner un modèle de classification traditionnel
X_train, y_train = extract_features(train_ds)
X_val, y_val = extract_features(val_ds)
X_test, y_test = extract_features(test_ds)

In [29]:
from sklearn.linear_model import LogisticRegression

# Création d'un modèle de régression logistique pour la classification multi-classe
log_reg = LogisticRegression(max_iter=2000)  # 2000 itérations max pour convergence

# Entraînement du modèle sur les features extraites
log_reg.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,2000
,multi_class,'deprecated'


In [30]:
from sklearn.calibration import CalibratedClassifierCV

# Calibration du modèle pour obtenir des probabilités plus fiables
# Platt scaling améliore la qualité des probabilités prédites
calibrated_model = CalibratedClassifierCV(
    log_reg,
    method="sigmoid",   # Platt scaling : méthode de calibration par fonction sigmoïde
    cv="prefit"         # Utilisation du modèle déjà entraîné (pas de cross-validation)
)

# Entraînement de la calibration sur le dataset de validation
calibrated_model.fit(X_val, y_val)

c:\Users\arthu\miniconda3\Lib\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


,estimator,LogisticRegre...max_iter=2000)
,method,'sigmoid'
,cv,'prefit'
,n_jobs,None
,ensemble,'auto'
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
